In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-07 02:19:57.899729: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_3_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

5/5 [==============================] - 0s 39ms/step - loss: 10.6385 - mean_squared_error: 10.6385 - rmse: 3.2617 - mean_absolute_error: 2.6254 - val_loss: 14.1104 - val_mean_squared_error: 14.1104 - val_rmse: 3.7564 - val_mean_absolute_error: 2.7545 - lr: 5.0000e-04
Epoch 24/1000
5/5 [==============================] - 0s 39ms/step - loss: 10.4827 - mean_squared_error: 10.4827 - rmse: 3.2377 - mean_absolute_error: 2.5698 - val_loss: 12.1951 - val_mean_squared_error: 12.1951 - val_rmse: 3.4922 - val_mean_absolute_error: 2.5541 - lr: 5.0000e-04
Epoch 25/1000
5/5 [==============================] - 0s 50ms/step - loss: 10.3025 - mean_squared_error: 10.3025 - rmse: 3.2098 - mean_absolute_error: 2.5517 - val_loss: 13.2058 - val_mean_squared_error: 13.2058 - val_rmse: 3.6340 - val_mean_absolute_error: 2.6601 - lr: 5.0000e-04
Epoch 26/1000
5/5 [==============================] - 0s 28ms/step - loss: 10.9468 - mean_squared_error: 10.9468 - rmse: 3.3086 - mean_absolute_error: 2.6285 - val_loss: 14

Epoch 52/1000
5/5 [==============================] - 0s 42ms/step - loss: 8.9451 - mean_squared_error: 8.9451 - rmse: 2.9908 - mean_absolute_error: 2.3730 - val_loss: 11.5024 - val_mean_squared_error: 11.5024 - val_rmse: 3.3915 - val_mean_absolute_error: 2.5324 - lr: 2.5000e-04
Epoch 53/1000
5/5 [==============================] - 0s 47ms/step - loss: 8.5981 - mean_squared_error: 8.5981 - rmse: 2.9323 - mean_absolute_error: 2.2983 - val_loss: 10.1063 - val_mean_squared_error: 10.1063 - val_rmse: 3.1790 - val_mean_absolute_error: 2.3740 - lr: 2.5000e-04
Epoch 54/1000
5/5 [==============================] - 0s 65ms/step - loss: 9.0651 - mean_squared_error: 9.0651 - rmse: 3.0108 - mean_absolute_error: 2.3648 - val_loss: 10.2044 - val_mean_squared_error: 10.2044 - val_rmse: 3.1944 - val_mean_absolute_error: 2.3975 - lr: 2.5000e-04
Epoch 55/1000
5/5 [==============================] - 0s 45ms/step - loss: 8.3160 - mean_squared_error: 8.3160 - rmse: 2.8838 - mean_absolute_error: 2.3452 - val_lo

5/5 [==============================] - 0s 61ms/step - loss: 7.3995 - mean_squared_error: 7.3995 - rmse: 2.7202 - mean_absolute_error: 2.1477 - val_loss: 7.7853 - val_mean_squared_error: 7.7853 - val_rmse: 2.7902 - val_mean_absolute_error: 2.1461 - lr: 2.5000e-04
Epoch 82/1000
5/5 [==============================] - 0s 53ms/step - loss: 7.5539 - mean_squared_error: 7.5539 - rmse: 2.7484 - mean_absolute_error: 2.1851 - val_loss: 7.0338 - val_mean_squared_error: 7.0338 - val_rmse: 2.6521 - val_mean_absolute_error: 2.0565 - lr: 2.5000e-04
Epoch 83/1000
5/5 [==============================] - 0s 36ms/step - loss: 7.8439 - mean_squared_error: 7.8439 - rmse: 2.8007 - mean_absolute_error: 2.2148 - val_loss: 7.6568 - val_mean_squared_error: 7.6568 - val_rmse: 2.7671 - val_mean_absolute_error: 2.1275 - lr: 2.5000e-04
Epoch 84/1000
5/5 [==============================] - 0s 59ms/step - loss: 7.6543 - mean_squared_error: 7.6543 - rmse: 2.7666 - mean_absolute_error: 2.2358 - val_loss: 12.3485 - val_me

Epoch 110/1000
5/5 [==============================] - 0s 46ms/step - loss: 6.2292 - mean_squared_error: 6.2292 - rmse: 2.4958 - mean_absolute_error: 1.9305 - val_loss: 7.8060 - val_mean_squared_error: 7.8060 - val_rmse: 2.7939 - val_mean_absolute_error: 2.1639 - lr: 1.2500e-04
Epoch 111/1000
5/5 [==============================] - 0s 51ms/step - loss: 6.5544 - mean_squared_error: 6.5544 - rmse: 2.5602 - mean_absolute_error: 2.0171 - val_loss: 9.1155 - val_mean_squared_error: 9.1155 - val_rmse: 3.0192 - val_mean_absolute_error: 2.3503 - lr: 1.2500e-04
Epoch 112/1000
5/5 [==============================] - 0s 51ms/step - loss: 6.4784 - mean_squared_error: 6.4784 - rmse: 2.5453 - mean_absolute_error: 1.9842 - val_loss: 7.3486 - val_mean_squared_error: 7.3486 - val_rmse: 2.7108 - val_mean_absolute_error: 2.0799 - lr: 1.2500e-04
Epoch 113/1000
5/5 [==============================] - 0s 56ms/step - loss: 6.8234 - mean_squared_error: 6.8234 - rmse: 2.6122 - mean_absolute_error: 2.0374 - val_loss

...

5/5 [==============================] - 0s 32ms/step - loss: 6.7695 - mean_squared_error: 6.7695 - rmse: 2.6018 - mean_absolute_error: 2.0740 - val_loss: 11.4810 - val_mean_squared_error: 11.4810 - val_rmse: 3.3884 - val_mean_absolute_error: 2.5634 - lr: 2.5000e-04
Epoch 135/1000
5/5 [==============================] - 0s 35ms/step - loss: 7.5545 - mean_squared_error: 7.5545 - rmse: 2.7485 - mean_absolute_error: 2.1017 - val_loss: 10.3914 - val_mean_squared_error: 10.3914 - val_rmse: 3.2236 - val_mean_absolute_error: 2.4243 - lr: 2.5000e-04
Epoch 136/1000
5/5 [==============================] - 0s 33ms/step - loss: 7.6349 - mean_squared_error: 7.6349 - rmse: 2.7631 - mean_absolute_error: 2.0965 - val_loss: 8.5574 - val_mean_squared_error: 8.5574 - val_rmse: 2.9253 - val_mean_absolute_error: 2.1942 - lr: 2.5000e-04
Epoch 137/1000
5/5 [==============================] - ETA: 0s - loss: 7.0928 - mean_squared_error: 7.0928 - rmse: 2.6632 - mean_absolute_error: 2.0829
Epoch 00137: ReduceLROnPla

Epoch 163/1000
5/5 [==============================] - 0s 38ms/step - loss: 6.5503 - mean_squared_error: 6.5503 - rmse: 2.5594 - mean_absolute_error: 2.0421 - val_loss: 8.6173 - val_mean_squared_error: 8.6173 - val_rmse: 2.9355 - val_mean_absolute_error: 2.2100 - lr: 1.2500e-04
Epoch 164/1000
5/5 [==============================] - 0s 35ms/step - loss: 6.6100 - mean_squared_error: 6.6100 - rmse: 2.5710 - mean_absolute_error: 2.0121 - val_loss: 8.7887 - val_mean_squared_error: 8.7887 - val_rmse: 2.9646 - val_mean_absolute_error: 2.2325 - lr: 1.2500e-04
Epoch 165/1000
5/5 [==============================] - 0s 37ms/step - loss: 6.3992 - mean_squared_error: 6.3992 - rmse: 2.5297 - mean_absolute_error: 1.9553 - val_loss: 8.7679 - val_mean_squared_error: 8.7679 - val_rmse: 2.9611 - val_mean_absolute_error: 2.2381 - lr: 1.2500e-04
Epoch 166/1000
5/5 [==============================] - 0s 43ms/step - loss: 6.3430 - mean_squared_error: 6.3430 - rmse: 2.5185 - mean_absolute_error: 1.9572 - val_loss

Epoch 192/1000
5/5 [==============================] - 0s 35ms/step - loss: 5.9345 - mean_squared_error: 5.9345 - rmse: 2.4361 - mean_absolute_error: 1.8683 - val_loss: 8.6991 - val_mean_squared_error: 8.6991 - val_rmse: 2.9494 - val_mean_absolute_error: 2.2336 - lr: 6.2500e-05
Epoch 193/1000
5/5 [==============================] - 0s 36ms/step - loss: 6.1664 - mean_squared_error: 6.1664 - rmse: 2.4832 - mean_absolute_error: 1.9132 - val_loss: 8.9373 - val_mean_squared_error: 8.9373 - val_rmse: 2.9895 - val_mean_absolute_error: 2.2661 - lr: 6.2500e-05
Epoch 194/1000
5/5 [==============================] - 0s 31ms/step - loss: 5.9302 - mean_squared_error: 5.9302 - rmse: 2.4352 - mean_absolute_error: 1.8861 - val_loss: 9.5219 - val_mean_squared_error: 9.5219 - val_rmse: 3.0858 - val_mean_absolute_error: 2.3464 - lr: 6.2500e-05
Epoch 195/1000
3/5 [=================>............] - ETA: 0s - loss: 7.0489 - mean_squared_error: 7.0489 - rmse: 2.6550 - mean_absolute_error: 2.0126
Epoch 00195: Re

Epoch 220/1000
5/5 [==============================] - 0s 36ms/step - loss: 6.0300 - mean_squared_error: 6.0300 - rmse: 2.4556 - mean_absolute_error: 1.9047 - val_loss: 9.0997 - val_mean_squared_error: 9.0997 - val_rmse: 3.0166 - val_mean_absolute_error: 2.2897 - lr: 1.5625e-05
Epoch 221/1000
5/5 [==============================] - 0s 29ms/step - loss: 6.2183 - mean_squared_error: 6.2183 - rmse: 2.4936 - mean_absolute_error: 1.9186 - val_loss: 8.9991 - val_mean_squared_error: 8.9991 - val_rmse: 2.9999 - val_mean_absolute_error: 2.2761 - lr: 1.5625e-05
Epoch 222/1000
5/5 [==============================] - 0s 25ms/step - loss: 5.8655 - mean_squared_error: 5.8655 - rmse: 2.4219 - mean_absolute_error: 1.9067 - val_loss: 9.1245 - val_mean_squared_error: 9.1245 - val_rmse: 3.0207 - val_mean_absolute_error: 2.2944 - lr: 1.5625e-05
Epoch 223/1000
5/5 [==============================] - 0s 25ms/step - loss: 6.0190 - mean_squared_error: 6.0190 - rmse: 2.4534 - mean_absolute_error: 1.8641 - val_loss

Epoch 249/1000
5/5 [==============================] - 0s 25ms/step - loss: 6.2509 - mean_squared_error: 6.2509 - rmse: 2.5002 - mean_absolute_error: 1.9405 - val_loss: 9.0864 - val_mean_squared_error: 9.0864 - val_rmse: 3.0144 - val_mean_absolute_error: 2.2891 - lr: 7.8125e-06
Epoch 250/1000
5/5 [==============================] - 0s 27ms/step - loss: 6.1182 - mean_squared_error: 6.1182 - rmse: 2.4735 - mean_absolute_error: 1.8840 - val_loss: 8.9283 - val_mean_squared_error: 8.9283 - val_rmse: 2.9880 - val_mean_absolute_error: 2.2673 - lr: 7.8125e-06
Epoch 251/1000
5/5 [==============================] - 0s 26ms/step - loss: 5.8349 - mean_squared_error: 5.8349 - rmse: 2.4155 - mean_absolute_error: 1.8593 - val_loss: 8.7242 - val_mean_squared_error: 8.7242 - val_rmse: 2.9537 - val_mean_absolute_error: 2.2387 - lr: 7.8125e-06
Epoch 252/1000
5/5 [==============================] - 0s 25ms/step - loss: 5.7018 - mean_squared_error: 5.7018 - rmse: 2.3878 - mean_absolute_error: 1.8752 - val_loss

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)